<a href="https://colab.research.google.com/github/helenason/CNN-shallowNet/blob/main/shallowNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
filname = '/content/gdrive/MyDrive/Colab Notebooks/data/fer2013.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names = ['emotion','pixels', 'usage']
df = pd.read_csv(filname,names=names, na_filter=False)
im = df['pixels']
df.head(10)
df.shape

(35888, 3)

In [4]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) / 255.0 for p in row[1].split()])

    X, Y = np.array(X), np.array(Y)
    return X, Y

In [5]:
X, Y = getData(filname)
print(X.shape)
print(Y.shape)
num_class = len(set(Y))
print(num_class)

(35887, 2304)
(35887,)
7


In [6]:
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [8]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
# from keras.layers.normalization import BatchNormalization
from keras.layers import BatchNormalization
# from keras import backend as K

In [9]:
# def my_model():
#     model = Sequential()
#     input_shape = (48,48,1)
#     model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
#     model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
#     model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
#     model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Flatten())
#     model.add(Dense(128))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(7))
#     model.add(Activation('softmax'))
    
#     model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
#     # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
#     #model.summary()
    
#     return model
    
# model=my_model()
# model.summary()

In [10]:
# INPUT => CONV => MaxPooling => RELU => FC 
def shallow_model():
    model = Sequential()
    input_shape = (48, 48, 1)
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Flatten())
    model.add(Dense(7)) # 7: 클래스 개수
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

model = shallow_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 7)                 258055    
                                                                 
 activation (Activation)     (None, 7)                 0         
                                                                 
Total params: 259,719
Trainable params: 259,719
Non-trainable params: 0
__________________________________________________

In [11]:
path_model='model_filter.h5' # save model at this location after each epoch
keras.backend.clear_session() # destroys the current graph and builds a new one
model = shallow_model() # create the model
keras.backend.set_value(model.optimizer.lr, 1e-3) # set the learning rate

h = model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
)

Epoch 1/20
505/505 [==============================] - 6s 6ms/step - loss: 1.6359 - accuracy: 0.3616 - val_loss: 1.5314 - val_accuracy: 0.4174
Epoch 2/20
505/505 [==============================] - 3s 6ms/step - loss: 1.4723 - accuracy: 0.4431 - val_loss: 1.4708 - val_accuracy: 0.4391
Epoch 3/20
505/505 [==============================] - 3s 5ms/step - loss: 1.3790 - accuracy: 0.4816 - val_loss: 1.4307 - val_accuracy: 0.4692
Epoch 4/20
505/505 [==============================] - 3s 5ms/step - loss: 1.3040 - accuracy: 0.5115 - val_loss: 1.4184 - val_accuracy: 0.4700
Epoch 5/20
505/505 [==============================] - 3s 5ms/step - loss: 1.2319 - accuracy: 0.5428 - val_loss: 1.4261 - val_accuracy: 0.4678
Epoch 6/20
505/505 [==============================] - 3s 6ms/step - loss: 1.1679 - accuracy: 0.5666 - val_loss: 1.4304 - val_accuracy: 0.4776
Epoch 7/20
505/505 [==============================] - 3s 6ms/step - loss: 1.1029 - accuracy: 0.5957 - val_loss: 1.4101 - val_accuracy: 0.4857
Epoch 

In [12]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

[0 1 2 3 4 5 6]


In [13]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))

    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()

In [14]:
y_pred=model.predict(X_test)
#print(y_pred)
y_test.shape

113/113 [==============================] - 0s 2ms/step


(3589, 7)

In [41]:
import keras.utils as image

def showResult(i) :

    img = image.load_img(i, color_mode = "grayscale", target_size=(48, 48))
    show_img=image.load_img(i, target_size=(200, 200))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)

    x /= 255

    custom = model.predict(x)
    # # print(custom[0])
    # emotion_analysis(custom[0])

    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);

    # plt.gray()
    # plt.imshow(show_img)
    # plt.show()

    idx = 0
    m = 0.000000000000000000001
    a = custom[0]
    for i in range(0,len(a)): # max 찾기
        if a[i]>m:
            m=a[i]
            idx=i
            
    # print('Expression Prediction:',objects[idx])

    if idx == 3 : # happy
    # if idx == 4 : # sad
        return 1
    else :
        return 0


import os

res = 0

address = '/content/gdrive/MyDrive/Colab Notebooks/data/archive/test/test/happy/'

images = os.listdir(address)

for img in images :
    res += showResult(address + img)

1/1 [==============================] - 0s 18ms/step


In [42]:
happy_res = res / 1774 * 100
print(happy_res) # 88%

88.50056369785794


In [38]:
sad_res = res / 1240 * 100
print(sad_res) # 73%

73.62903225806451
